In [5]:
def convert_to_conf(file_path):
    file = open(file_path, "r") 
    arr_lines = file.readlines() 

    separator = ","
    i=0
    for line in arr_lines:
        line = line.rstrip()

        if(i == len(arr_lines) - 1):
            separator = ""
        pre_white = ' ' * (len(line) - len(line.lstrip()))
        print pre_white+'"'+line.lstrip()+'"'+separator
        i= i + 1

In [6]:
#convert the text to array elements to put inside the configuration file
#copy and paste the printed text inside the config file

#convert_to_conf("query_doi_search.txt")
convert_to_conf("query-test.txt")
#convert_to_conf("query_orcid_search.txt")

"select ?work ?title ?authorname where {",
  "?work rdfs:label ?lbl1 .",
  "?work wdt:P31 wd:Q13442814.",
  "FILTER (langMatches( lang(?lbl1), "EN" ) )",
  "FILTER( regex(?lbl1, 'machine' ))",
  "?work wdt:P31 wd:Q13442814.",
  "optional { ?work wdt:P1476 ?title .}",
  "optional {",
    "?work wdt:P50 ?author .",
    "?author rdfs:label ?authorname .",
    "FILTER (langMatches( lang(?authorname), "EN" ) )",
  "}",
  "BIND(REPLACE(STR(?work), 'http://www.wikidata.org/', '', 'i') as ?short_iri) .",
  "optional { ?work wdt:P1433 ?venue . }",
  "optional { ?work wdt:P577 ?date . }",
  "optional { ?work wdt:P478 ?volume . }",
  "optional { ?work wdt:P433 ?issue . }",
  "optional { ?work wdt:P304 ?pages . }",
  "optional { ?work wdt:P275 ?license_ .}",
  "optional { ?work wdt:P356 ?doi . }",
  "optional { ?work wdt:P953 ?url . }",
"}",
"LIMIT 500"


In [11]:
#reverse conversion

arr_str = [
   "select ?work ?short_iri ?title ?venueLabel  (STR(YEAR (?alldate)) as ?date) ?volume ?author ?authorname ?issue ?pages ?license ?doi ?url (COUNT(distinct ?cited_by) AS ?in_cits) ?type where {",
          "?work rdfs:label ?lbl1 .",
          "?work wdt:P31 wd:Q13442814.",
          "FILTER (langMatches( lang(?lbl1), 'EN' ) )",
          "FILTER( regex(?lbl1, '[[VAR]]' ))",
          "?work wdt:P31 wd:Q13442814.",
          "optional { ?work wdt:P1476 ?title .}",
          "optional {",
              "?work wdt:P50 ?author .",
              "?author rdfs:label ?authorname .",
              "FILTER (langMatches( lang(?authorname), 'EN' ) )",
          "}",
          "BIND(REPLACE(STR(?work), 'http://www.wikidata.org/', '', 'i') as ?short_iri) .",
          "optional { ?work wdt:P1433 ?venue . }",
          "optional { ?cited_by wdt:P2860 ?work.}",
          "optional { ?work wdt:P577 ?alldate . }",
          "optional { ?work wdt:P478 ?volume . }",
          "optional { ?work wdt:P433 ?issue . }",
          "optional { ?work wdt:P304 ?pages . }",
          "optional { ?work wdt:P275 ?license_ .}",
          "optional { ?work wdt:P356 ?doi . }",
          "optional { ?work wdt:P953 ?url . }",
            "}",
            "GROUP BY ?work ?short_iri ?title ?venueLabel ?date ?volume ?author ?authorname ?issue ?pages ?license ?doi ?url ?type ",
            "LIMIT 200"
]


In [12]:
for entry in arr_str:
    entry = entry.replace('"', '')
    print entry

select ?work ?short_iri ?title ?venueLabel  (STR(YEAR (?alldate)) as ?date) ?volume ?author ?authorname ?issue ?pages ?license ?doi ?url (COUNT(distinct ?cited_by) AS ?in_cits) ?type where {
?work rdfs:label ?lbl1 .
?work wdt:P31 wd:Q13442814.
FILTER (langMatches( lang(?lbl1), 'EN' ) )
FILTER( regex(?lbl1, '[[VAR]]' ))
?work wdt:P31 wd:Q13442814.
optional { ?work wdt:P1476 ?title .}
optional {
?work wdt:P50 ?author .
?author rdfs:label ?authorname .
FILTER (langMatches( lang(?authorname), 'EN' ) )
}
BIND(REPLACE(STR(?work), 'http://www.wikidata.org/', '', 'i') as ?short_iri) .
optional { ?work wdt:P1433 ?venue . }
optional { ?cited_by wdt:P2860 ?work.}
optional { ?work wdt:P577 ?alldate . }
optional { ?work wdt:P478 ?volume . }
optional { ?work wdt:P433 ?issue . }
optional { ?work wdt:P304 ?pages . }
optional { ?work wdt:P275 ?license_ .}
optional { ?work wdt:P356 ?doi . }
optional { ?work wdt:P953 ?url . }
}
GROUP BY ?work ?short_iri ?title ?venueLabel ?date ?volume ?author ?authornam

In [7]:
import re

#convert 1 line string to SPARQL query
str_line = " PREFIX cito:<http://purl.org/spar/cito/>  PREFIX dcterms:<http://purl.org/dc/terms/>  PREFIX datacite:<http://purl.org/spar/datacite/>  PREFIX literal:<http://www.essepuntato.it/2010/06/literalreification/>  PREFIX biro:<http://purl.org/spar/biro/>  PREFIX frbr:<http://purl.org/vocab/frbr/core#>  PREFIX c4o:<http://purl.org/spar/c4o/>  PREFIX bds:<http://www.bigdata.com/rdf/search#>  PREFIX fabio:<http://purl.org/spar/fabio/>  PREFIX pro:<http://purl.org/spar/pro/>  PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>  SELECT DISTINCT ?iri ?short_iri ?doi ?title ?year ?score ?author ?author_iri (COUNT(distinct ?cited) AS ?out_cits) (COUNT(distinct ?cited_by) AS ?in_cits) WHERE  {  { BIND('machine learning' as ?free_txt) . ?lit bds:search ?free_txt . ?lit bds:matchAllTerms 'true' . ?lit bds:relevance ?score . ?lit bds:minRelevance '0.2' . ?lit bds:maxRank '300' . { ?entry c4o:hasContent ?lit . ?entry biro:references ?iri . } UNION {?iri dcterms:title  ?lit } UNION {?iri fabio:hasSubtitle ?lit} .} UNION { BIND('10.1002/cncr.28414' as ?doi_txt) . ?lit bds:search ?doi_txt . ?lit bds:matchAllTerms 'true' . ?lit bds:relevance ?score . ?lit bds:maxRank '1' . ?iri datacite:hasIdentifier/literal:hasLiteralValue ?lit .} OPTIONAL { ?iri rdf:type ?type . BIND(REPLACE(STR(?iri), 'https://w3id.org/oc/corpus', '', 'i') as ?short_iri) . OPTIONAL {?iri dcterms:title  ?title .} OPTIONAL {?iri fabio:hasSubtitle  ?subtitle .} OPTIONAL {?iri fabio:hasPublicationYear ?year .} OPTIONAL {?iri cito:cites ?cited .} OPTIONAL {?cited_by cito:cites ?iri .} OPTIONAL { ?iri datacite:hasIdentifier [ datacite:usesIdentifierScheme datacite:doi ; literal:hasLiteralValue ?doi ] }  OPTIONAL { ?iri pro:isDocumentContextFor [ pro:withRole pro:author ; pro:isHeldBy ?author_iri ]. ?author_iri foaf:familyName ?fname . ?author_iri foaf:givenName ?name . BIND(CONCAT(STR(?name),' ', STR(?fname)) as ?author) . } } }GROUP BY ?iri ?short_iri ?doi ?title ?year ?score ?author ?author_iri  ORDER BY DESC(?score)  LIMIT 2000"

print re.sub('\s{2,}', ' \n', str_line)




 PREFIX cito:<http://purl.org/spar/cito/> 
PREFIX dcterms:<http://purl.org/dc/terms/> 
PREFIX datacite:<http://purl.org/spar/datacite/> 
PREFIX literal:<http://www.essepuntato.it/2010/06/literalreification/> 
PREFIX biro:<http://purl.org/spar/biro/> 
PREFIX frbr:<http://purl.org/vocab/frbr/core#> 
PREFIX c4o:<http://purl.org/spar/c4o/> 
PREFIX bds:<http://www.bigdata.com/rdf/search#> 
PREFIX fabio:<http://purl.org/spar/fabio/> 
PREFIX pro:<http://purl.org/spar/pro/> 
PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
SELECT DISTINCT ?iri ?short_iri ?doi ?title ?year ?score ?author ?author_iri (COUNT(distinct ?cited) AS ?out_cits) (COUNT(distinct ?cited_by) AS ?in_cits) WHERE 
{ 
{ BIND('machine learning' as ?free_txt) . ?lit bds:search ?free_txt . ?lit bds:matchAllTerms 'true' . ?lit bds:relevance ?score . ?lit bds:minRelevance '0.2' . ?lit bds:maxRank '300' . { ?entry c4o:hasContent ?lit . ?entry biro:references ?iri . } UNION {?iri dcterms:title 
?lit } UNION {?iri fabio:hasSu